# Import related functions

In [1]:
import re
import sys
sys.path.append(r'./script/')
from ECMpy_function import *
from cobra.core import Reaction,Metabolite

# Input and output files

In [2]:
ori_sbml_path = "./data/iCW773.xml"
json_path = "./model/iCW773.json"
cg_gene_id = pd.read_csv('./data/cg_gene_id_trans.csv', index_col='cgl_id')
cg_model_gene_information = pd.read_csv("./data/cg_model_gene_information.csv",index_col='ID')
gpr_modifications = pd.read_csv('./data/gpr_modifications.tsv', index_col=0,sep='\t')
gene_subnum_path = "./data/gene_subnum.csv"

json_path_output="./model/iCW773_gpr.json"
sbml_path_output="./model/iCW773_gpr.xml"
json_path_final_output="./model/iCW773_gpr_uniprot.json"
sbml_path_final_output="./model/iCW773_gpr_uniprot.xml"
json_path_final_output_del="./model/iCW773_gpr_uniprot_del.json"
sbml_path_final_output_del="./model/iCW773_gpr_uniprot_del.xml"
json_split_file = "./data/iCW773_uniprot_split.json"
json_subnum_out_file = "./data/reaction_gene_subunit_num.json"

# Initial model change

### The model iCW773 from the supplemental data of Zhang et al. (https://biotechnologyforbiofuels.biomedcentral.com/articles/10.1186/s13068-017-0856-3), modified metabolite chemical formula and converted the model to XML format.   
### To meet the requirements of AutoPACMEN and ECMpy processes for metabolic network format input, we modified the gene, reaction and metabolite information in the model as follows: 
### (1) Metabolites correction: ‘(e)’ to ‘_e’, ‘-D’ to ‘__D’, ‘-L’ to ‘__L’, ‘-R’ to ‘__R’ and other ‘-‘ to ‘_’.   
### (2) Reaction’s correction: ‘-’ to ‘__’ in reactions beginning with ‘EX’ and ‘-’ to ‘__’ in other reactions.   

# model modification

In [3]:
model=cobra.io.read_sbml_model(ori_sbml_path)
#change gpr
for eachr in model.reactions:
    if eachr.id in list(gpr_modifications.index):
        if str(gpr_modifications.loc[eachr.id,'gpr']) == 'nan':
            eachr.gene_reaction_rule = ''
        else:
            eachr.gene_reaction_rule=gpr_modifications.loc[eachr.id,'gpr']
    
cobra.io.write_sbml_model(model,sbml_path_output)    
cobra.io.save_json_model(model, json_path_output)

# Adding UniProt ID information in the annotation, which is the basis for obtaining kinetic parameters.

In [4]:
model=cobra.io.read_sbml_model(sbml_path_output)
      
for eachr in model.reactions: 
    if re.search(' or ',eachr.gene_reaction_rule):
        genelist=eachr.gene_reaction_rule.split(' or ')
        newgenelist=[]
        for eachgene in genelist:
            if re.search(' and ',eachgene):
                eachgene = str(eachgene).strip('()')
                eachgene = str(eachgene).strip(' ')
                genegenelist=eachgene.split(' and ')
                newnewgenelist=[]
                for eacheachgene in genegenelist:
                    if eacheachgene in cg_model_gene_information.index:
                        newnewgenelist.append(cg_model_gene_information.loc[eacheachgene,'Cgl_ID'])
                    else:
                        newnewgenelist.append(eacheachgene)
                eachgene =' and '.join(newnewgenelist)
                if len(eachgene) > 11:
                    eachgene = '('+eachgene+')'
                if re.search('Cgl0236',eachgene):
                    eachgene = 'Cgl0236 and Cgl1293'
                newgenelist.append(eachgene)
            else:
                if eachgene in cg_model_gene_information.index:
                    newgenelist.append(cg_model_gene_information.loc[eachgene,'Cgl_ID'])   
                else:
                    newgenelist.append(eachgene)
        if len(newgenelist)>1:
            eachr.gene_reaction_rule=' or '.join(newgenelist)
    elif re.search(' and ',eachr.gene_reaction_rule):
        genelist=eachr.gene_reaction_rule.split(' and ')
        newgenelist=[]
        for eachgene in genelist:
            if eachgene in cg_model_gene_information.index:
                newgenelist.append(cg_model_gene_information.loc[eachgene,'Cgl_ID'])
            else:
                newgenelist.append(eachgene)
        if len(newgenelist)>1:
            eachr.gene_reaction_rule=' and '.join(newgenelist)
    else:
        if eachr.gene_reaction_rule in cg_model_gene_information.index:
            eachr.gene_reaction_rule=cg_model_gene_information.loc[eachr.gene_reaction_rule,'Cgl_ID']

for eachmet in model.metabolites: 
    if re.search('_c',eachmet.id):
        eachmet.compartment='c'
    elif re.search('_p',eachmet.id):
        eachmet.compartment='p'
    elif re.search('_e',eachmet.id):
        eachmet.compartment='e'
        
for eachgene in model.genes: 
    if eachgene.id in cg_model_gene_information.index:
        eachgene.id=cg_model_gene_information.loc[eachgene.id,'Cgl_ID']
        eachgene.name='G_'+eachgene.id
        eachgene.annotation={'uniprot': cg_gene_id.loc[eachgene.id,'Entry']}
    elif re.search('Cgl',eachgene.id):
        eachgene.name='G_'+eachgene.id
        eachgene.annotation={'uniprot': cg_gene_id.loc[eachgene.id,'Entry']}   
        
cobra.io.write_sbml_model(model,sbml_path_final_output)
cobra.io.save_json_model(model,json_path_final_output)
len(model.genes)

1470

# Delete duplicated gene 

In [5]:
# del duplicated gene 
import re
import copy
import ast
dictionary_model=json_load(json_path_final_output)


gene_dic = []
for eachg in dictionary_model['genes']:
    gene_dic.append(str(eachg))
df = pd.DataFrame({'gene':gene_dic})
df.drop_duplicates(keep='first',inplace=True)
list1 = df['gene'].tolist()


dictionary_model['genes'].clear()
list1 = ','.join(str(i) for i in list1)
dictionary_model['genes'] = str(list1)
dictionary_model['genes'] = ast.literal_eval(dictionary_model['genes'])


json_write(json_path_final_output_del,dictionary_model)
json_model=cobra.io.load_json_model(json_path_final_output_del)
cobra.io.write_sbml_model(json_model,sbml_path_final_output_del)
print(len(json_model.genes))

779


# Generate subunit number json file

In [6]:
# split model
model = cobra.io.json.load_json_model(json_path_final_output_del)
convert_to_irreversible(model)
model = isoenzyme_split(model)
cobra.io.save_json_model(model, json_split_file)

In [7]:
# extract subunit numbers by reaction
dictionary_model=json_load(json_split_file)
gene_subnum = pd.read_csv(gene_subnum_path)
gene_subnum.set_index(['keggNames'], inplace=True)

reaction_gene_subnum = {}
for eachr in dictionary_model['reactions']: 
    model_id_list = []
    model_name_list = []

    if len(eachr['gene_reaction_rule']) != 0 :
        if re.search(' and ',eachr['gene_reaction_rule']):
            gene_subnum_dict = {}
            newgenelist=[]

            genelist=eachr['gene_reaction_rule'].split(' and ')

            for eachgene in genelist:
                if eachgene in gene_subnum.index:
                    subnum = gene_subnum.loc[eachgene,'subunitnumber']
                    reaction_gene_subnum.setdefault(eachr['id'], {}).update({eachgene : str(subnum)})

        else:
            gene_subnum_dict = {}
            if eachr['gene_reaction_rule'] in gene_subnum.index:
                subnum = gene_subnum.loc[eachr['gene_reaction_rule'],'subunitnumber']
                gene_subnum_dict[str(eachr['gene_reaction_rule'])]= str(subnum)
                reaction_gene_subnum[eachr['id']] = gene_subnum_dict
        
json_write(json_subnum_out_file,reaction_gene_subnum)
